<a href="https://colab.research.google.com/github/soutrik71/ERAV2_Advanced_VisionTransformers/blob/main/notebooks/AutoEncoderBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will be used to explore in details about Auto Encoder and its uses with both Cifar and MNIST datasets.
- Basic Implementation of AE in pytorch
- Implement AE using pytorch lightning

In [2]:
import os
import torch
from torch.utils.data import DataLoader,Dataset,random_split,SubsetRandomSampler,RandomSampler
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
latent_dims = 512
num_epoch = 50
batch_size = 64
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Data loading Application

In [4]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [6]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=img_transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=img_transform, download=True)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
for x,y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


#### Model Definition

In [9]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size = 4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size = 4, stride=2, padding=1)
        self.linear = nn.Linear(in_features = 64*7*7, out_features = latent_dims)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x